#### Copyright 2019, Autores do TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação em dados desequilibrados

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/imbalanced_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver no TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar caderno</a></td>
</table>

Este tutorial demonstra como classificar um conjunto de dados altamente desequilibrado no qual o número de exemplos em uma classe supera em muito os exemplos em outra. Você trabalhará com o conjunto de dados de [detecção de fraude de cartão de crédito](https://www.kaggle.com/mlg-ulb/creditcardfraud) hospedado no Kaggle. O objetivo é detectar apenas 492 transações fraudulentas de um total de 284.807 transações. Você usará [Keras](../../guide/keras/overview.ipynb) para definir o modelo e [os pesos das classes](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model) para ajudar o modelo a aprender com os dados desequilibrados. .

Este tutorial contém o código completo para:

- Carregue um arquivo CSV usando o Pandas.
- Crie conjuntos de treinamento, validação e teste.
- Defina e treine um modelo usando Keras (incluindo a definição de pesos de classe).
- Avalie o modelo usando várias métricas (incluindo precisão e recall).
- Experimente técnicas comuns para lidar com dados desequilibrados, como:
    - Ponderação da classe
    - Sobreamostragem


## Configuração

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Processamento e exploração de dados

### Baixe o conjunto de dados de fraude de cartão de crédito Kaggle

Pandas é uma biblioteca Python com muitos utilitários úteis para carregar e trabalhar com dados estruturados e pode ser usado para baixar CSVs em um dataframe.

Note: This dataset has been collected and analysed during a research collaboration of Worldline and the [Machine Learning Group](http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available [here](https://www.researchgate.net/project/Fraud-detection-5) and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

In [ ]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

### Examine o desequilíbrio do rótulo da classe

Vejamos o desequilíbrio do conjunto de dados:

In [ ]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Isso mostra a pequena fração de amostras positivas.

### Limpe, divida e normalize os dados

Os dados brutos têm alguns problemas. Primeiro, as colunas `Time` e `Amount` são muito variáveis para serem usadas diretamente. Elimine a coluna `Time` (já que não está claro o que significa) e pegue o log da coluna `Amount` para reduzir seu intervalo.

In [ ]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps=0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

Divida o conjunto de dados em conjuntos de treinamento, validação e teste. O conjunto de validação é usado durante o ajuste do modelo para avaliar a perda e quaisquer métricas, no entanto, o modelo não se ajusta a esses dados. O conjunto de teste não é usado durante a fase de treinamento e só é usado no final para avaliar quão bem o modelo generaliza para novos dados. Isso é especialmente importante com conjuntos de dados desequilibrados, onde o [overfitting](https://developers.google.com/machine-learning/crash-course/generalization/peril-of-overfitting) é uma preocupação significativa devido à falta de dados de treinamento.

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

Normalize os recursos de entrada usando o sklearn StandardScaler. Isso definirá a média como 0 e o desvio padrão como 1.

Nota: O `StandardScaler` só se encaixa usando `train_features` para ter certeza de que o modelo não está espiando os conjuntos de validação ou teste. 

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)


Cuidado: Se você deseja implantar um modelo, é fundamental preservar os cálculos de pré-processamento. A maneira mais fácil de implementá-los como camadas e anexá-los ao seu modelo antes de exportar.


### Observe a distribuição de dados

Em seguida, compare as distribuições dos exemplos positivos e negativos em alguns recursos. Boas perguntas a se fazer neste momento são:

- Essas distribuições fazem sentido?
    - Sim. Você normalizou a entrada e estes estão principalmente concentrados na faixa `+/- 2` .
- Você pode ver a diferença entre as distribuições?
    - Sim, os exemplos positivos contêm uma taxa muito mais alta de valores extremos.

In [ ]:
pos_df = pd.DataFrame(train_features[ bool_train_labels], columns = train_df.columns)
neg_df = pd.DataFrame(train_features[~bool_train_labels], columns = train_df.columns)

sns.jointplot(pos_df['V5'], pos_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(neg_df['V5'], neg_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
_ = plt.suptitle("Negative distribution")

## Defina o modelo e as métricas

Definir uma função que cria uma rede neural simples com uma camada densly conectado escondida, um [abandono](https://developers.google.com/machine-learning/glossary/#dropout_regularization) camada para reduzir overfitting, e uma camada sigmóide saída que retorna a probabilidade de uma transação ser fraudulenta: 

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def make_model(metrics = METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

### Compreender métricas úteis

Observe que existem algumas métricas definidas acima que podem ser calculadas pelo modelo e que serão úteis na avaliação do desempenho.

- **Falsos** negativos e **falsos** positivos são amostras que foram classificadas **incorretamente**
- **Verdadeiros** negativos e **verdadeiros** positivos são amostras que foram classificadas **corretamente**
- **A precisão** é a porcentagem de exemplos classificados corretamente

> $ \ frac {\ text {amostras verdadeiras}} {\ text {amostras totais}} $

- **A precisão** é a porcentagem de positivos **previstos** que foram classificados corretamente

> $ \ frac {\ text {verdadeiros positivos}} {\ text {verdadeiros positivos + falsos positivos}} $

- **Lembre-se** é a porcentagem de positivos **reais** que foram classificados corretamente

> $ \ frac {\ text {verdadeiros positivos}} {\ text {verdadeiros positivos + falsos negativos}} $

- **AUC** refere-se à área sob a curva de uma curva de característica de operação do receptor (ROC-AUC). Essa métrica é igual à probabilidade de que um classificador classifique uma amostra aleatória positiva mais alta do que uma amostra aleatória negativa.

Observação: a precisão não é uma métrica útil para essa tarefa. Você pode obter 99,8% + de precisão nesta tarefa prevendo False o tempo todo.

Consulte Mais informação:

- [Verdadeiro x falso e positivo x negativo](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
- [Precisão](https://developers.google.com/machine-learning/crash-course/classification/accuracy)
- [Precisão e recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)
- [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

## Modelo de linha de base

### Construir o modelo

Agora crie e treine seu modelo usando a função que foi definida anteriormente. Observe que o modelo é ajustado usando um tamanho de lote maior do que o padrão de 2048, isso é importante para garantir que cada lote tenha uma chance decente de conter algumas amostras positivas. Se o tamanho do lote fosse muito pequeno, eles provavelmente não teriam transações fraudulentas com as quais aprender.

Observação: este modelo não lidará bem com o desequilíbrio de classe. Você irá aprimorá-lo posteriormente neste tutorial.

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = make_model()
model.summary()

Teste o modelo:

In [ ]:
model.predict(train_features[:10])

### Opcional: defina a polarização inicial correta.

Essas suposições iniciais não são boas. Você sabe que o conjunto de dados está desequilibrado. Defina o viés da camada de saída para refletir isso (consulte: [Uma receita para o treinamento de redes neurais: "init well"](http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines) ). Isso pode ajudar na convergência inicial.

Com a inicialização de polarização padrão, a perda deve ser de cerca de `math.log(2) = 0.69314` 

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

A polarização correta a ser definida pode ser derivada de:

$$ p_0 = pos / (pos + neg) = 1 / (1 + e ^ {- b_0}) $$ $$ b_0 = -log_e (1 / p_0 - 1) $$ $$ b_0 = log_e (pos / neg ) $$

In [ ]:
initial_bias = np.log([pos/neg])
initial_bias

Defina isso como o viés inicial e o modelo dará suposições iniciais muito mais razoáveis.

Deve estar próximo a: `pos/total = 0.0018`

In [ ]:
model = make_model(output_bias = initial_bias)
model.predict(train_features[:10])

Com esta inicialização, a perda inicial deve ser de aproximadamente:

$$ - p_0log (p_0) - (1-p_0) log (1-p_0) = 0,01317 $$

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Essa perda inicial é cerca de 50 vezes menor do que teria ocorrido com a inicialização ingênua.

Dessa forma, o modelo não precisa passar as primeiras épocas apenas aprendendo que exemplos positivos são improváveis. Isso também facilita a leitura dos gráficos de perda durante o treinamento.

### Verifique os pesos iniciais

Para tornar as várias execuções de treinamento mais comparáveis, mantenha os pesos deste modelo inicial em um arquivo de checkpoint e carregue-os em cada modelo antes do treinamento.

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
model.save_weights(initial_weights)

### Confirme se a correção de viés ajuda

Antes de prosseguir, confirme rapidamente se a inicialização de polarização cuidadosa realmente ajudou.

Treine o modelo por 20 épocas, com e sem essa inicialização cuidadosa, e compare as perdas: 

In [ ]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
def plot_loss(history, label, n):
  # Use a log scale to show the wide range of values.
  plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
  plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  
  plt.legend()

In [ ]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

A figura acima deixa claro: Em termos de perda de validação, neste problema, esta inicialização cuidadosa dá uma vantagem clara. 

### Treine o modelo

In [ ]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels))

### Verifique o histórico de treinamento

Nesta seção, você produzirá gráficos da precisão e perda do seu modelo no conjunto de treinamento e validação. Eles são úteis para verificar se há overfitting, sobre o qual você pode aprender mais neste [tutorial](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) .

Além disso, você pode produzir esses gráficos para qualquer uma das métricas criadas acima. Os falsos negativos são incluídos como exemplo.

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()


In [ ]:
plot_metrics(baseline_history)

Observação: que a curva de validação geralmente tem um desempenho melhor do que a curva de treinamento. Isso é causado principalmente pelo fato de que a camada de dropout não está ativa ao avaliar o modelo.

### Avalie as métricas

Você pode usar uma [matriz de confusão](https://developers.google.com/machine-learning/glossary/#confusion_matrix) para resumir os rótulos reais e previstos, em que o eixo X é o rótulo previsto e o eixo Y é o rótulo real.

In [ ]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

Avalie seu modelo no conjunto de dados de teste e exiba os resultados para as métricas que você criou acima.

In [ ]:
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)

Se o modelo tivesse previsto tudo perfeitamente, esta seria uma [matriz diagonal](https://en.wikipedia.org/wiki/Diagonal_matrix) onde os valores fora da diagonal principal, indicando previsões incorretas, seriam zero. Nesse caso, a matriz mostra que você tem relativamente poucos falsos positivos, o que significa que havia relativamente poucas transações legítimas que foram sinalizadas incorretamente. No entanto, você provavelmente desejaria ter ainda menos falsos negativos, apesar do custo de aumentar o número de falsos positivos. Essa troca pode ser preferível porque os falsos negativos permitiriam a realização de transações fraudulentas, ao passo que os falsos positivos podem fazer com que um e-mail seja enviado a um cliente solicitando a verificação da atividade do cartão.

### Trace o ROC

Agora plote o [ROC](https://developers.google.com/machine-learning/glossary#ROC) . Este gráfico é útil porque mostra, de relance, a faixa de desempenho que o modelo pode atingir apenas ajustando o limite de saída.

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

Parece que a precisão é relativamente alta, mas o recall e a área sob a curva ROC (AUC) não são tão altas quanto você gostaria. Os classificadores geralmente enfrentam desafios ao tentar maximizar a precisão e a recuperação, o que é especialmente verdadeiro quando se trabalha com conjuntos de dados desequilibrados. É importante considerar os custos dos diferentes tipos de erros no contexto do problema com o qual você se preocupa. Neste exemplo, um falso negativo (uma transação fraudulenta é perdida) pode ter um custo financeiro, enquanto um falso positivo (uma transação é sinalizada incorretamente como fraudulenta) pode diminuir a felicidade do usuário.

## Pesos de classe

### Calcular pesos de classe

O objetivo é identificar transações fraudulentas, mas você não tem muitas dessas amostras positivas para trabalhar, então você gostaria que o classificador pesasse muito os poucos exemplos que estão disponíveis. Você pode fazer isso passando pesos Keras para cada classe por meio de um parâmetro. Isso fará com que o modelo "preste mais atenção" aos exemplos de uma classe sub-representada.

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

### Treine um modelo com pesos de classe

Agora tente treinar novamente e avaliar o modelo com pesos de classe para ver como isso afeta as previsões.

Nota: Usar `class_weights` muda o intervalo da perda. Isso pode afetar a estabilidade do treinamento dependendo do otimizador. Otimizadores cujo tamanho do passo depende da magnitude do gradiente, como `optimizers.SGD` , podem falhar. O otimizador usado aqui, `optimizers.Adam` , não é afetado pela mudança de escala. Observe também que, devido à ponderação, as perdas totais não são comparáveis entre os dois modelos.

In [ ]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight) 

### Verifique o histórico de treinamento

In [ ]:
plot_metrics(weighted_history)

### Avalie as métricas

In [ ]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

Aqui você pode ver que, com pesos de classe, a exatidão e a precisão são menores porque há mais falsos positivos, mas, inversamente, a recuperação e a AUC são maiores porque o modelo também encontrou mais positivos verdadeiros. Apesar de ter menor precisão, este modelo tem maior recall (e identifica mais transações fraudulentas). Obviamente, há um custo para os dois tipos de erro (você também não gostaria de incomodar os usuários sinalizando muitas transações legítimas como fraudulentas). Considere cuidadosamente as compensações entre esses diferentes tipos de erros para seu aplicativo.

### Trace o ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right')

## Sobreamostragem

### Superamostrar a classe minoritária

Uma abordagem relacionada seria reamostrar o conjunto de dados sobreamostrando a classe minoritária.

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

#### Usando NumPy

Você pode equilibrar o conjunto de dados manualmente, escolhendo o número certo de índices aleatórios dos exemplos positivos:

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

#### Usando `tf.data`

Se você estiver usando `tf.data` a maneira mais fácil de produzir exemplos equilibrados é começar com um conjunto de dados `positive` e um `negative` e mesclá-los. Veja [o guia tf.data](../../guide/data.ipynb) para mais exemplos.

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

Cada conjunto de dados fornece pares `(feature, label)` :

In [ ]:
for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

Junte os dois usando `experimental.sample_from_datasets` :

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

Para usar este conjunto de dados, você precisará do número de etapas por época.

A definição de "época", neste caso, é menos clara. Digamos que seja o número de lotes necessários para ver cada exemplo negativo uma vez:

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

### Treine nos dados sobreamostrados

Agora tente treinar o modelo com o conjunto de dados reamostrado em vez de usar pesos de classe para ver como esses métodos se comparam.

Nota: Como os dados foram balanceados replicando os exemplos positivos, o tamanho total do conjunto de dados é maior e cada época executa mais etapas de treinamento. 

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks = [early_stopping],
    validation_data=val_ds)

Se o processo de treinamento considerasse todo o conjunto de dados em cada atualização de gradiente, essa sobreamostragem seria basicamente idêntica à ponderação da classe.

Mas ao treinar o modelo em lote, como você fez aqui, os dados sobreamostrados fornecem um sinal de gradiente mais suave: em vez de cada exemplo positivo sendo mostrado em um lote com um grande peso, eles são mostrados em muitos lotes diferentes a cada vez com um peso pequeno.

Este sinal de gradiente mais suave torna mais fácil treinar o modelo.

### Verifique o histórico de treinamento

Observe que as distribuições de métricas serão diferentes aqui, porque os dados de treinamento têm uma distribuição totalmente diferente dos dados de validação e teste. 

In [ ]:
plot_metrics(resampled_history )

### Re-treinar


Como o treinamento é mais fácil com os dados balanceados, o procedimento de treinamento acima pode se ajustar rapidamente.

So break up the epochs to give the `callbacks.EarlyStopping` finer control over when to stop training.

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs
    steps_per_epoch = 20,
    epochs=10*EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_ds))

### Verifique novamente o histórico de treinamento

In [ ]:
plot_metrics(resampled_history)

### Avalie as métricas

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)

### Trace o ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_roc("Train Resampled", train_labels, train_predictions_resampled,  color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled,  color=colors[2], linestyle='--')
plt.legend(loc='lower right')

## Aplicando este tutorial ao seu problema

A classificação de dados desequilibrada é uma tarefa inerentemente difícil, pois há tão poucos exemplos para aprender. Você deve sempre começar com os dados primeiro e fazer o seu melhor para coletar o máximo de amostras possível e dar uma ideia substancial sobre quais recursos podem ser relevantes para que o modelo possa obter o máximo de sua classe minoritária. Em algum ponto, seu modelo pode ter dificuldades para melhorar e produzir os resultados desejados, portanto, é importante ter em mente o contexto do seu problema e as compensações entre os diferentes tipos de erros.